# <ins><font color=#2360F0>How to use MUSTARD estimator</font></ins>

<font color='red'>A script version of this notebook is available [here](https://github.com/Sand-jrd/mustard/blob/main/demo.py) </font>

<img src="../example-data/demo.gif" alt="demo" width="500" text-align="center"/>

## <ins><font color=#1945D6>Build the estimator</font></ins>

MUSTARD is object-oriented.
The estimator object need the ADI cube <font color='red'>**centred and normalized**</font> and the angle to be build... and that is it !

Nevertherless, here is a non-exhaustive list of optional parameters that can be handy
- `coro` : the size of the coronographe mask in pixels. (default is 6)
- `pupil` : the size of the pupil in pixels. (default is `"edge"`). If you want to estimate the full matrix you can set it to `None`
- `Badframes` : frames that will be ignore (0 is the first frame).



In [1]:
from vip_hci.fits import open_fits
from mustard import mustard_estimator

# Load the data 
datadir = "./example-data/"
science_data = open_fits(datadir+"cube.fits")
angles = open_fits(datadir+"angles.fits")

# Build the estimator
estimator =  mustard_estimator(science_data, angles, coro=8, pupil="edge")


Fits HDU-0 data successfully loaded. Data shape: (66, 256, 256)
Fits HDU-0 data successfully loaded. Data shape: (66,)


## <ins><font color=#1945D6>Use the estimator

Using the MUSTARD estimator is pretty straightforward.

### <ins><font color=#3498DB> First let's define the parameters for the processing

In [2]:
param = {'w_r'   : 0.05,      # Proportion of Regul over J
        'w_r2'   : 0.10,      # Proportion of Regul2 over J
        'w_way'  : (1, 0),    # You can either work with derotated_cube or rotated cube. Or both
        'gtol'   : 1e-7,      # Gradient tolerence. Stop the estimation when the mean of gradient will hit the value
        'kactiv' : 3,         # Iter before activate regul (i.e when to compute true weight base on w_r proportion)
        'estimI' : "Both",    # Estimate frames flux is highly recommended ! possible value : {"Frame","L","Both"}
        'med_sub': True,      # perform a median substraction highly recommended !
        'weighted_rot' : True,# Compute weight for each frame according to PA angle separations.
        'suffix' : "",        # Name of your simulation (this is optional)
        'res_pos': False,     # Penalize negative residual
        'maxiter': 60}        # Maximum number of iterations (it converge fast tbh)


There is a bunch of other options. Those presented below are the most important.

-`w_r` and `w_r2`, define respectivly the weight of the regularization terms to smooth and to sort circular ambiguity (the last one will be explain later). <font color='red'>**A weight between 10%-5% is recommanded**</font> in order to <ins>let the data attachment terme drive the estimation</ins> to avoid deformations. R2 can be higher if you purposly want to get rid of all circular ambiguities from the disk estimations. 

-`w_r` define the model definition. **If your signal is corrupted by Gibbs artifact, you can swhitch to reverse mode.** In reverse mode, the speckels map is rotating and it is less likely to cause such problem. You can also try both way at the same time but it takes more time and will not necessarly give better results. 

-`estimI`, see [Frame Amplitudes](#Frame-Amplitues)


#### <ins><font color=#3498DB>Define the regularization (optional)

If you are not satified with the aspect of the result, you can modify your prior through the regularization.

##### <ins><font color=#17A67E>R2 : For circular ambiguities

The main problem of the MUSTARD approch is the inherent ambiguities of the model. This ambiguities can be resolved with prior. 

The more obvious ambiguity is circular shaped signal beacause it can be assigned both in the static or rotating part. 
**TIP** : Checkout the MUSTARD result [here](#Results), you will clearly see the halo of circular ambiguities in the speackel map. 

You can configure a regularization to sort how you want these ambiguies to be resovle with the method `configR2`.
I recommand `mode='l1'` and `penaliz="X"` or a `mode='mask'` with a mask that penalize more at the edge of the coronographic mask than the rest of the frame as shown below.

In [3]:
from mustard.utils import ellipse, circle, gaussian 

# Create a mask
shape = estimator.model.frame_shape # Get the frame shape here.
M = circle(shape, shape[0]//2) + 10*circle(shape, 13) 

# Configure R2
estimator.configR2(Msk=None, mode="l1", penaliz="X", invert=True)


#### <ins><font color=#17A67E> R2 : Smooth

There is some *"experimental"* smooth mode that is do not recommand.
The only relevant option is `p_L=0.5`. It define the weight of the smooth regularization of the speackel map over the rotating signal map.

In [4]:
estimator.configR1(mode="smooth", p_L=0.5)

### <ins><font color=#3498DB> Start the minimization !

You are now ready. Stating the minimization is one line.
The parameters we defined in a dictionnary above will be used here.
There is also some extra parameters :
- save : if you provide a path, it will save the results as fits file there
- verbode : to print information in the console
- gif : to create a gif of you simulation, if you want to see the evolution trought the iteration.

The demo from "exemple-data" usually takes ~4mins.
    
**TIP :** Click on the ◼︎ icon to stop prematurely the minimization
(or ctrl+C if you are using a console). It will quit proprely the iterative loop. Results will be stored/saved/return.


In [ ]:
L_est, X_est, flux =  estimator.estimate(**param, save=datadir,
                                                  gif=False,
                                                  verbose=True,
                                                  history=True)

__________________________________________________
Resolving IP-ADI optimization problem - name : 
 Outputs will be saved ./example-data/
Regul R1 : 'smooth' and R2 : 'l1 on X inverted'
No deconvolution and with frame weighted based on rotations
Relative amplitude of BothX and L will be estimated
Regul weight are set to w_r=5.00e-02 and w_r2=1.00e-01, maxiter=60

|it |       loss        |        R1        |        R2        |       Rpos       |       total      |
| 0 |6.503639e-05 (100%)|0.000000e+00 (0 %)|0.000000e+00 (0 %)|0.000000e+00 (0 %)|6.503639112875e-05|


**INFO** : The iteration k+1/2 is the step where regularization weights are computed. 

### <ins><font color=#3498DB>Results

The estimator store all the results. It does also provide some method to get relevant information.

In [ ]:
import matplotlib.pyplot as plt
from numpy import percentile

plt.figure("Results",figsize=(16,9))
plt.subplot(121), plt.imshow(L_est,cmap='jet',vmax=percentile(L_est,99.5))
plt.subplot(122), plt.imshow(X_est,cmap='jet',vmax=percentile(L_est,99))


#### <ins><font color=#17A67E> Evolution of the criteria
This method return the array of the values of the criteria at each iteration. If you set `show=True` or `save=$path$`, it will also plot the convergence curve.

In [ ]:
evo = estimator.get_evo_convergence()

##### <ins><font color=#17A67E>Frame Amplitues

The relatives amplitudes is the how the amplitudes varies between frame. The amplitudes of the rotating signal and the speackles are estiamted separelty. The amplitudes of the first frame is 1. Hence the **two vectors returned by the fuction have have the size NumberOfFrames-1**

The function below return the two vector of frame amplitude `ampL, ampX`.
If you set `show=True` or `save=$path$`, it will also plot the relative difference of amplitude of each frame.

In [ ]:
flx = estimator.get_flux()

#### <ins><font color=#17A67E> Frame weight 

The frame weight is computed based on the PA angle vector of each frame. Indeed for the sake of the estimation, if the angle between two frame is two small, there is chance that the signal will overllap and it can biais the results.

The fuction below return the vector of frame weights.
If you set `show=True` or `save=$path$`, it will also plot the weight bars and the PA angle cuve.

In [ ]:
wr = estimator.get_rot_weight()

#### <ins><font color=#17A67E> Residual

This is the noise. If you see some structures that is alright. This map containt every kind of noise - not only white.
MUSTARD can distangle noise that have a life time between *( exposure time < lifetime < aquisition-time/2 )*.

**Look carefully**, because if you see very clear structes that look like the disk and/or that appear on more that half of the frames it might be a sign that something went wrong.


In [ ]:
from numpy import percentile
from hciplot import plot_cubes

residual = estimator.get_residual()
lim = percentile(abs(residual),99) # TIP : try without percentile.
plot_cubes(residual,cmap="jet",vmax=lim,vmin=-lim)


#### <ins><font color=#17A67E> Reconstruction

This is the reconstruction. It should look like the initial ADI cube.

In [ ]:
reconstruction = estimator.get_reconstruction()
plot_cubes(reconstruction, cmap="jet", vmax=percentile(reconstruction,99))

**BONUS** : You can generate the mustard gif of your simulation (the one in introcution)

In [ ]:
estimator.mustard_results()

The result will be print saved in your datadir. 